In [1]:
import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt

direc = 'C:/Users/54115/Documents/Ramiro/Taller'

# 1. Price Dataframe ETL

In [2]:
price = pd.read_csv(direc + '/BTC_USD Bitfinex Historical Data.csv', 
                    sep = ',', 
                    usecols = [0,1,2,3,4], 
                    names = ['date', 'close', 'open', 'high', 'low'],
                    header = 0)

price['close'] = price['close'].str.replace(',', '').astype(float)
price['open'] = price['open'].str.replace(',', '').astype(float)
price['high'] = price['high'].str.replace(',', '').astype(float)
price['low'] = price['low'].str.replace(',', '').astype(float)

price['date'] = pd.to_datetime(price['date'], format='%m/%d/%Y')

price = price.sort_values('date', ascending=True)

price.reset_index(inplace = True, drop=True)

In [3]:
price

,date,close,open,high,low
0,2018-01-01,13354.0,13794.0,13893.0,12787.0
1,2018-01-02,14709.8,13448.0,15300.0,12810.0
2,2018-01-03,15155.0,14713.0,15428.0,14522.0
3,2018-01-04,15160.0,15150.0,15390.0,14064.0
4,2018-01-05,16917.0,15159.0,17101.0,14769.0
...,...,...,...,...,...
1977,2023-06-01,26833.0,27220.0,27348.0,26666.0
1978,2023-06-02,27254.0,26833.0,27301.0,26523.0
1979,2023-06-03,27077.0,27254.0,27331.0,26925.0
1980,2023-06-04,27126.0,27077.0,27446.0,26963.0


In [4]:
def date_filter(df, start_date, end_date):
    "Filters the price dataframe and restarts the index"
    
    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]
    
    filtered_df.reset_index(inplace = True, drop = True)
    
    return filtered_df 

# 2. Price Indicators

In [5]:
def calculate_sma(dataframe, n):
    """
    Calculate the simple moving average (SMA) of the 'close' price for each date using n periods.
    
    Args:
        dataframe (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods to calculate the SMA.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'sma_n' containing the calculated SMA.
    """
    
    name = 'sma_' + str(n)
    
    dataframe[name] = dataframe['close'].rolling(window=n).mean()
    return dataframe

In [6]:
def calculate_ema(dataframe, n):
    """
    Calculate the exponential moving average (EMA) of the 'close' price for each date using n periods.
    
    Args:
        dataframe (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods to calculate the EMA.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'ema_n' containing the calculated EMA.
    """
    
    name = 'ema_' + str(n)
    
    dataframe[name] = dataframe['close'].ewm(span=n, adjust=False).mean()
    return dataframe

In [7]:
def calculate_boll(dataframe, n, o):
    """
    Calculate the bollinger bands of the 'close' price for each date using n periods.
    
    Args:
        dataframe (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods to calculate the bollinger bands.
        o (int): Number of standard deviations to calculate the bollinger bands.
    
    Returns:
        pd.DataFrame: DataFrame with 3 additional columns 'li', 'lc', 'ls', 
        containing the bollinger bands.
    """
    
    name_li = 'li'
    name_lc = 'lc'
    name_ls = 'ls'
    
    dataframe[name_li] = dataframe['close'].rolling(window=n).mean()-2*dataframe['close'].rolling(window=n).std()
    dataframe[name_lc] = dataframe['close'].rolling(window=n).mean()
    dataframe[name_ls] = dataframe['close'].rolling(window=n).mean()+2*dataframe['close'].rolling(window=n).std()
    return dataframe

In [8]:
def calculate_macd(dataframe, n, m, s):
    """
    Calculate the MACD (Moving Average Convergence Divergence) line, signal line, and histogram line.
    
    Args:
        dataframe (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods for the short-term EMA (Exponential Moving Average).
        m (int): Number of periods for the long-term EMA.
        s (int): Number of periods for the signal line EMA.
    
    Returns:
        pd.DataFrame: DataFrame with additional columns 'macd_line', 'signal_line', 
        and 'histogram_line' containing the calculated values.
    """
    dataframe['short_ema'] = dataframe['close'].ewm(span=n, adjust=False).mean()
    dataframe['long_ema'] = dataframe['close'].ewm(span=m, adjust=False).mean()
    dataframe['macd_line'] = dataframe['short_ema'] - dataframe['long_ema']
    dataframe['signal_line'] = dataframe['macd_line'].ewm(span=s, adjust=False).mean()
    dataframe['histogram_line'] = dataframe['macd_line'] - dataframe['signal_line']
    return dataframe

In [9]:
def calculate_rsi(dataframe, n):
    """
    Calculate the Relative Strength Index (RSI) using n periods.
    
    Args:
        dataframe (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods to calculate the RSI.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'rsi_n' containing the calculated RSI values.
    """
    price_diff = dataframe['close'].diff()  # Calculate price difference between consecutive periods
    up_prices = price_diff.where(price_diff > 0, 0)  # Store positive price differences as up prices
    down_prices = -price_diff.where(price_diff < 0, 0)  # Store negative price differences as down prices

    avg_gain = up_prices.rolling(window=n).mean()  # Calculate average gain over n periods
    avg_loss = down_prices.rolling(window=n).mean()  # Calculate average loss over n periods

    relative_strength = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + relative_strength))
    
    name = 'rsi_' + str(n)
    
    dataframe[name] = rsi
    return dataframe

# 3. Plot Price Indicators

In [10]:
def generate_candlestick_chart(dataframe, start_date, end_date):
    """
    Generate a candlestick chart using a copy of the 'open', 'close', 'low', 'high', and 'date' columns of the DataFrame,
    within the specified start and end date range. Green candles represent close price higher than open price,
    while red candles represent close price lower than open price.
    
    Args:
        dataframe (pd.DataFrame): Input DataFrame containing 'open', 'close', 'low', 'high', and 'date' columns.
        start_date (str): Start date in the format 'YYYY-MM-DD'.
        end_date (str): End date in the format 'YYYY-MM-DD'.
    """
    # Create a copy of the input DataFrame
    filtered_data = dataframe.copy()

    # Convert the 'date' column to a pandas datetime object
    filtered_data['date'] = pd.to_datetime(filtered_data['date'])

    # Set the 'date' column as the index of the DataFrame
    filtered_data.set_index('date', inplace=True)

    # Filter the DataFrame based on the start and end dates
    filtered_data = filtered_data.loc[start_date:end_date]

    # Define the style of the candlestick plot
    style = mpf.make_mpf_style(base_mpf_style='classic', marketcolors=mpf.make_marketcolors(up='g', down='r'))

    # Create the candlestick plot
    mpf.plot(filtered_data, type='candle', style=style, ylabel='Precio (USD)', xlabel='Fecha')
    
    # Save the plot
    

    # Display the plot
    mpf.show()

In [11]:
"""Example: 
generate_candlestick_chart(price, '2022-01-01', '2022-03-30')"""

"Example: \ngenerate_candlestick_chart(price, '2022-01-01', '2022-03-30')"

In [12]:
def chart_sma(df, start_date, end_date, n, m):
    plt.figure()
    fig, ax = plt.subplots(1, gridspec_kw={'height_ratios': [3]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = calculate_sma(filtered_df, n)
    name_n = 'sma_' + str(n)
    filtered_df = calculate_sma(filtered_df, m)
    name_m = 'sma_' + str(m)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax.plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de  \n Cierre')
    ax.plot(filtered_df['date'], filtered_df[name_n], color='orange', label='SMA (' + str(n) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_m], color='purple', label='SMA (' + str(m) + ')')
    ax.grid(True)
    ax.set_ylabel('Precio (USD)')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Fecha')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
        
    filename = direc + '/sma_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [13]:
"""Example:
chart_sma(price, '2022-01-01', '2022-03-30', 14, 28)"""

"Example:\nchart_sma(price, '2022-01-01', '2022-03-30', 14, 28)"

In [14]:
def chart_ema(df, start_date, end_date, n, m):
    plt.figure()
    fig, ax = plt.subplots(1, gridspec_kw={'height_ratios': [3]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = calculate_ema(filtered_df, n)
    name_n = 'ema_' + str(n)
    filtered_df = calculate_ema(filtered_df, m)
    name_m = 'ema_' + str(m)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax.plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax.plot(filtered_df['date'], filtered_df[name_n], color='orange', label='EMA (' + str(n) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_m], color='purple', label='EMA (' + str(m) + ')')
    ax.grid(True)
    ax.set_ylabel('Precio (USD)')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Fecha')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
        
    filename = direc + '/ema_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [15]:
"""Example:
chart_ema(price, '2022-01-01', '2022-03-30', 14, 28)"""

"Example:\nchart_ema(price, '2022-01-01', '2022-03-30', 14, 28)"

In [16]:
def chart_sma_ema(df, start_date, end_date, n):
    plt.figure()
    fig, ax = plt.subplots(1, gridspec_kw={'height_ratios': [3]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = calculate_sma(filtered_df, n)
    name_n = 'sma_' + str(n)
    filtered_df = calculate_ema(filtered_df, n)
    name_m = 'ema_' + str(n)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax.plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax.plot(filtered_df['date'], filtered_df[name_n], color='orange', label='SMA (' + str(n) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_m], color='purple', label='EMA (' + str(n) + ')')
    ax.grid(True)
    ax.set_ylabel('Precio (USD)')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Fecha')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
        
    filename = direc + '/sma_ema_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [17]:
"""Example:
chart_sma_ema(price, '2022-01-01', '2022-03-30', 25)"""

"Example:\nchart_sma_ema(price, '2022-01-01', '2022-03-30', 25)"

In [18]:
def chart_boll(df, start_date, end_date, n, o):
    plt.figure()
    fig, ax = plt.subplots(1, gridspec_kw={'height_ratios': [3]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = calculate_boll(filtered_df, n, o)
    name_li = 'li'
    name_lc = 'lc'
    name_ls = 'ls'
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax.plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax.plot(filtered_df['date'], filtered_df[name_lc], color='orange', label='LC (' + str(n) + ',' + str(o) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_li], color='purple', label='LI (' + str(n) + ',' + str(o) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_ls], color='purple', label='LS (' + str(n) + ',' + str(o) + ')')
    ax.grid(True)
    ax.set_ylabel('Precio (USD)')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Fecha')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
        
    filename = direc + '/boll_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [19]:
"""Example:
chart_boll(price, '2022-01-01', '2022-03-30', 21, 2)"""

"Example:\nchart_boll(price, '2022-01-01', '2022-03-30', 21, 2)"

In [20]:
def chart_rsi(df, start_date, end_date, n, l, h):
    plt.figure()
    fig, ax = plt.subplots(2, gridspec_kw={'height_ratios': [3, 1]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = calculate_rsi(filtered_df, n)
    name = 'rsi_' + str(n)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax[0].plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax[0].grid(True)
    ax[0].set_ylabel('Precio (USD)')
    ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # Remove x-axis tick labels in ax[0]
    ax[0].set_xticklabels([])

    # Plot RSI chart in ax[1]
    ax[1].plot(filtered_df['date'], filtered_df[name], label='RSI (' + str(n) + ')')
    ax[1].set_ylim(0, 100)
    ax[1].axhline(y=h, color='r', linestyle='-', label='Sobrecomprado (' + str(h) + ')')
    ax[1].axhline(y=l, color='r', linestyle='-', label='Sobrevendido (' + str(l) + ')')
    ax[1].grid(True)
    ax[1].set_ylabel('RSI')
    ax[1].set_xlabel('Fecha')
    for label in ax[1].get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    filename = direc + '/rsi_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [21]:
"""Example:
chart_rsi(price, '2022-01-01', '2022-03-30', 14, 25, 75)"""

"Example:\nchart_rsi(price, '2022-01-01', '2022-03-30', 14, 25, 75)"

In [22]:
def chart_macd(df, start_date, end_date, n, m, s):
    plt.figure()
    fig, ax = plt.subplots(2, gridspec_kw={'height_ratios': [3, 1]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = calculate_macd(filtered_df, n, m, s)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax[0].plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax[0].grid(True)
    ax[0].set_ylabel('Precio (USD)')
    ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # Remove x-axis tick labels in ax[0]
    ax[0].set_xticklabels([])

    # Plot MACD chart in ax[1]
    ax[1].plot(filtered_df['date'], filtered_df['macd_line'], label='MACD (' + str(n) + ',' + str(m) + ')')
    ax[1].plot(filtered_df['date'], filtered_df['signal_line'], label='Señal (' + str(s) + ')')
    ax[1].bar(filtered_df['date'], filtered_df['histogram_line'], width=0.5, label='Histograma')
    ax[1].grid(True)
    ax[1].set_xlabel('Fecha')
    for label in ax[1].get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    filename = direc + '/macd_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [23]:
"""Example:
chart_macd(price, '2022-01-01', '2022-03-30', 12, 26, 9)"""

"Example:\nchart_macd(price, '2022-01-01', '2022-03-30', 12, 26, 9)"

# 4. Generate and Plot the signals for each Algorithmic Strategy

In [24]:
def signal_sma(df, n, m):
    """
    Generate signals based on the crossover of two SMAs (Simple Moving Averages) of different time frames.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods for the shorter SMA.
        m (int): Number of periods for the longer SMA.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'signal' containing the generated signals.
    """
    # Calculate the SMAs
    df = calculate_sma(df, n)  # Shorter SMA (sma_n)
    name_n = 'sma_' + str(n)
    df = calculate_sma(df, m)  # Longer SMA (sma_m)
    name_m = 'sma_' + str(m)
    
    # Initialize the 'signal' column with NaN values
    df['signal'] = 0
    
    # Generate buy and sell signals
    sig = -1
    
    for i in range(1, len(df)):
        if df[name_n].iloc[i] > df[name_m].iloc[i] and df[name_n].iloc[i-1] < df[name_m].iloc[i-1] and sig == -1:
            df.at[i, 'signal'] = 1  # Buy signal
            sig = 1
        elif df[name_n].iloc[i] < df[name_m].iloc[i] and df[name_n].iloc[i-1] > df[name_m].iloc[i-1] and sig == 1:
            df.at[i, 'signal'] = -1  # Sell signal
            sig = -1
    
    return df

In [25]:
def chart_sma_signal(df, start_date, end_date, n, m):
    plt.figure()
    fig, ax = plt.subplots(1, gridspec_kw={'height_ratios': [3]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    #filtered_df = calculate_sma(filtered_df, n)
    name_n = 'sma_' + str(n)
    #filtered_df = calculate_sma(filtered_df, m)
    name_m = 'sma_' + str(m)
    filtered_df = signal_sma(filtered_df,n,m)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax.plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de  \n Cierre')
    ax.plot(filtered_df['date'], filtered_df[name_n], color='orange', label='SMA (' + str(n) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_m], color='purple', label='SMA (' + str(m) + ')')
    ax.grid(True)
    ax.set_ylabel('Precio (USD)')
    ax.set_xlabel('Fecha')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    
    # Add circles for buy and sell signals
    buy_dates = filtered_df[filtered_df['signal'] == 1]['date']
    sell_dates = filtered_df[filtered_df['signal'] == -1]['date']
    ax.scatter(buy_dates, filtered_df.loc[filtered_df['signal'] == 1, 'close'],
               marker='o', s=100, color='green', label='Señal de Compra')
    ax.scatter(sell_dates, filtered_df.loc[filtered_df['signal'] == -1, 'close'],
               marker='o', s=100, color='red', label='Señal de Venta')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    filename = direc + '/sma_signal_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [26]:
"""Example:
chart_sma_signal(price, '2019-10-01', '2020-06-30', 14, 28)"""

"Example:\nchart_sma_signal(price, '2019-10-01', '2020-06-30', 14, 28)"

In [27]:
def signal_ema(df, n, m):
    """
    Generate signals based on the crossover of two EMAs (Exponential Moving Averages) of different time frames.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods for the shorter EMA.
        m (int): Number of periods for the longer EMA.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'signal' containing the generated signals.
    """
    # Calculate the EMAs
    df = calculate_ema(df, n)  # Shorter EMA (ema_n)
    name_n = 'ema_' + str(n)
    df = calculate_ema(df, m)  # Longer EMA (ema_m)
    name_m = 'ema_' + str(m)
    
    # Initialize the 'signal' column with NaN values
    df['signal'] = 0
    
    # Generate buy and sell signals
    sig = -1
    
    for i in range(1, len(df)):
        if df[name_n].iloc[i] > df[name_m].iloc[i] and df[name_n].iloc[i-1] < df[name_m].iloc[i-1] and sig == -1:
            df.at[i, 'signal'] = 1  # Buy signal
            sig = 1
        elif df[name_n].iloc[i] < df[name_m].iloc[i] and df[name_n].iloc[i-1] > df[name_m].iloc[i-1] and sig == 1:
            df.at[i, 'signal'] = -1  # Sell signal
            sig = -1
    
    return df

In [28]:
def chart_ema_signal(df, start_date, end_date, n, m):
    plt.figure()
    fig, ax = plt.subplots(1, gridspec_kw={'height_ratios': [3]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    #filtered_df = calculate_ema(filtered_df, n)
    name_n = 'ema_' + str(n)
    #filtered_df = calculate_ema(filtered_df, m)
    name_m = 'ema_' + str(m)
    filtered_df = signal_ema(filtered_df,n,m)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax.plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de  \n Cierre')
    ax.plot(filtered_df['date'], filtered_df[name_n], color='orange', label='EMA (' + str(n) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_m], color='purple', label='EMA (' + str(m) + ')')
    ax.grid(True)
    ax.set_ylabel('Precio (USD)')
    ax.set_xlabel('Fecha')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    
    # Add circles for buy and sell signals
    buy_dates = filtered_df[filtered_df['signal'] == 1]['date']
    sell_dates = filtered_df[filtered_df['signal'] == -1]['date']
    ax.scatter(buy_dates, filtered_df.loc[filtered_df['signal'] == 1, 'close'],
               marker='o', s=100, color='green', label='Señal de Compra')
    ax.scatter(sell_dates, filtered_df.loc[filtered_df['signal'] == -1, 'close'],
               marker='o', s=100, color='red', label='Señal de Venta')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    filename = direc + '/ema_signal_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [29]:
"""Example:
chart_ema_signal(price, '2019-10-01', '2020-06-30', 21, 55)"""

"Example:\nchart_ema_signal(price, '2019-10-01', '2020-06-30', 21, 55)"

In [30]:
def signal_boll(df, n, o):
    """
    Generate signals based on the crossover of the closing price with the Bollinger Bands
    
    Args:
        df (pd.DataFrame): Input DataFrame containing 'date' and 'close' columns.
        n (int): Number of periods for the BB.
        o (float): Number of standard deviations to calculate the BB.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'signal' containing the generated signals.
    """
    # Calculate the BB
    df = calculate_boll(df, n, o)
    name_li = 'li'
    name_ls = 'ls'
    
    # Initialize the 'signal' column with NaN values
    df['signal'] = 0
    
    # Generate buy and sell signals
    sig = -1
    
    for i in range(1, len(df)):
        if df['close'].iloc[i] > df[name_li].iloc[i] and df['close'].iloc[i-1] <= df[name_li].iloc[i-1] and sig == -1:
            df.at[i, 'signal'] = 1  # Buy signal
            sig = 1
        elif df['close'].iloc[i] < df[name_ls].iloc[i] and df['close'].iloc[i-1] >= df[name_ls].iloc[i-1] and sig == 1:
            df.at[i, 'signal'] = -1  # Sell signal
            sig = -1
    
    return df

In [31]:
def chart_boll_signal(df, start_date, end_date, n, o):
    plt.figure()
    fig, ax = plt.subplots(1, gridspec_kw={'height_ratios': [3]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = signal_boll(filtered_df, n, o)
    name_li = 'li'
    name_lc = 'lc'
    name_ls = 'ls'
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax.plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax.plot(filtered_df['date'], filtered_df[name_lc], color='orange', label='LC (' + str(n) + ',' + str(o) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_li], color='purple', label='LI (' + str(n) + ',' + str(o) + ')')
    ax.plot(filtered_df['date'], filtered_df[name_ls], color='purple', label='LS (' + str(n) + ',' + str(o) + ')')
    ax.grid(True)
    ax.set_ylabel('Precio (USD)')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Fecha')
    for label in ax.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
        
    # Add circles for buy and sell signals
    buy_dates = filtered_df[filtered_df['signal'] == 1]['date']
    sell_dates = filtered_df[filtered_df['signal'] == -1]['date']
    ax.scatter(buy_dates, filtered_df.loc[filtered_df['signal'] == 1, 'close'],
               marker='o', s=100, color='green', label='Señal de Compra')
    ax.scatter(sell_dates, filtered_df.loc[filtered_df['signal'] == -1, 'close'],
               marker='o', s=100, color='red', label='Señal de Venta')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        
    filename = direc + '/boll_signal_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [32]:
"""Example:
chart_boll_signal(price, '2019-10-01', '2020-06-30', 21, 2)"""

"Example:\nchart_boll_signal(price, '2019-10-01', '2020-06-30', 21, 2)"

In [33]:
def signal_rsi(df, n, l, h):
    """
    Generate signals based on the crossover of RSI (Relative Strength Index) with low (oversold) and high (overbought) lines.
    
    Args:
        df (pd.DataFrame): Input DataFrame containing 'date', 'close', and 'rsi' columns.
        n (int): Number of periods for calculating the RSI.
        low_line (float): The low line value for generating buy signals.
        high_line (float): The high line value for generating sell signals.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'signal' containing the generated signals.
    """
    # Calculate the RSI
    df = calculate_rsi(df, n)  # RSI (rsi_n)
    name_n = 'rsi_' + str(n)
    
    # Initialize the 'signal' column
    df['signal'] = 0
    
    # Generate buy and sell signals
    sig = -1
    
    for i in range(1, len(df)):
        if df[name_n].iloc[i] > l and df[name_n].iloc[i-1] <= l and sig == -1:
            df.at[i, 'signal'] = 1  # Buy signal
            sig = 1
        elif df[name_n].iloc[i] < h and df[name_n].iloc[i-1] >= h and sig == 1:
            df.at[i, 'signal'] = -1  # Sell signal
            sig = -1
    
    return df

In [34]:
def chart_rsi_signal(df, start_date, end_date, n, l, h):
    plt.figure()
    fig, ax = plt.subplots(2, gridspec_kw={'height_ratios': [3, 1]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    #filtered_df = calculate_rsi(filtered_df, n)
    name = 'rsi_' + str(n)
    filtered_df = signal_rsi(filtered_df,n,l,h)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax[0].plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax[0].grid(True)
    ax[0].set_ylabel('Precio (USD)')

    # Remove x-axis tick labels in ax[0]
    ax[0].set_xticklabels([])
    
    # Add circles for buy and sell signals in ax[0]
    buy_dates = filtered_df[filtered_df['signal'] == 1]['date']
    sell_dates = filtered_df[filtered_df['signal'] == -1]['date']
    ax[0].scatter(buy_dates, filtered_df.loc[filtered_df['signal'] == 1, 'close'],
               marker='o', s=100, color='green', label='Señal de Compra')
    ax[0].scatter(sell_dates, filtered_df.loc[filtered_df['signal'] == -1, 'close'],
               marker='o', s=100, color='red', label='Señal de Venta')
    ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Plot RSI chart in ax[1]
    ax[1].plot(filtered_df['date'], filtered_df[name], label='RSI (' + str(n) + ')')
    ax[1].set_ylim(0, 100)
    ax[1].axhline(y=h, color='r', linestyle='-', label='Sobrecomprado (' + str(h) + ')')
    ax[1].axhline(y=l, color='r', linestyle='-', label='Sobrevendido (' + str(l) + ')')
    ax[1].grid(True)
    ax[1].set_ylabel('RSI')
    ax[1].set_xlabel('Fecha')
    for label in ax[1].get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    filename = direc + '/rsi_signal_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [35]:
"""Example:
chart_rsi_signal(price, '2020-09-01', '2021-04-30', 14, 30, 70)"""

"Example:\nchart_rsi_signal(price, '2020-09-01', '2021-04-30', 14, 30, 70)"

In [36]:
def signal_macd(df, n, m, s):
    """
    Generate signals based on the crossover of the MACD Lines
    
    Args:
        df (pd.DataFrame): Input DataFrame containing 'date', 'close', and 'rsi' columns.
        n (int): Number of periods for the short-term EMA (Exponential Moving Average).
        m (int): Number of periods for the long-term EMA.
        s (int): Number of periods for the signal line EMA.
    
    Returns:
        pd.DataFrame: DataFrame with an additional column 'signal' containing the generated signals.
    """
    # Calculate the MACD
    df = calculate_macd(df, n, m, s)
    
    # Initialize the 'signal' column
    df['signal'] = 0
    
    # Generate buy and sell signals
    sig = -1
    
    for i in range(1, len(df)):
        if df['histogram_line'].iloc[i] > 0 and df['histogram_line'].iloc[i-1] <= 0 and sig == -1:
            df.at[i, 'signal'] = 1  # Buy signal
            sig = 1
        elif df['histogram_line'].iloc[i] < 0 and df['histogram_line'].iloc[i-1] >= 0 and sig == 1:
            df.at[i, 'signal'] = -1  # Sell signal
            sig = -1
    
    return df

In [37]:
def chart_macd_signal(df, start_date, end_date, n, m, s):
    plt.figure()
    fig, ax = plt.subplots(2, gridspec_kw={'height_ratios': [3, 1]})

    # Filter DataFrame based on start_date and end_date
    filtered_df = df.copy()
    filtered_df = signal_macd(filtered_df,n,m,s)
    filtered_df = filtered_df[filtered_df['date'] >= start_date]
    filtered_df = filtered_df[filtered_df['date'] <= end_date]

    # Plot line chart of 'close' column in ax[0] (price)
    ax[0].plot(filtered_df['date'], filtered_df['close'], color='black', label='Precio de \n Cierre')
    ax[0].grid(True)
    ax[0].set_ylabel('Precio (USD)')

    # Remove x-axis tick labels in ax[0]
    ax[0].set_xticklabels([])
    
    # Add circles for buy and sell signals in ax[0]
    buy_dates = filtered_df[filtered_df['signal'] == 1]['date']
    sell_dates = filtered_df[filtered_df['signal'] == -1]['date']
    ax[0].scatter(buy_dates, filtered_df.loc[filtered_df['signal'] == 1, 'close'],
               marker='o', s=100, color='green', label='Señal de Compra')
    ax[0].scatter(sell_dates, filtered_df.loc[filtered_df['signal'] == -1, 'close'],
               marker='o', s=100, color='red', label='Señal de Venta')
    ax[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    # Plot MACD chart in ax[1]
    ax[1].plot(filtered_df['date'], filtered_df['macd_line'], label='MACD (' + str(n) + ',' + str(m) + ')')
    ax[1].plot(filtered_df['date'], filtered_df['signal_line'], label='Señal (' + str(s) + ')')
    ax[1].bar(filtered_df['date'], filtered_df['histogram_line'], width=0.5, label='Histograma')
    ax[1].grid(True)
    ax[1].set_xlabel('Fecha')
    for label in ax[1].get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    ax[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    filename = direc + '/macd_signal_plot.png'
    plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [38]:
"""#Example:
chart_macd_signal(price, '2020-09-01', '2021-04-30', 12, 26, 9)"""

"#Example:\nchart_macd_signal(price, '2020-09-01', '2021-04-30', 12, 26, 9)"

# 5. Calculate a Portfolio using the signals

In [39]:
def calculate_portfolio(df):
    """
    Calculate the evolution of the portfolio and the portfolio metrics
    using the price and the signals of the strategy
    """
    
    # Create the portfolio dataframe
    portfolio = pd.DataFrame()
    portfolio['date'] = df['date']
    portfolio['close'] = df['close']
    portfolio['amount_btc'] = 0
    portfolio['amount_usd'] = 10000
    portfolio['value'] = 10000
    portfolio['min_value'] = 10000
    
    for i in range(1, len(df)):
        if df['signal'].iloc[i] == 1: 
            portfolio.loc[i, 'amount_btc'] = portfolio.loc[i-1, 'amount_usd'] / portfolio.loc[i, 'close'] # Buy BTC
            portfolio.loc[i, 'amount_usd'] = 0
        elif df['signal'].iloc[i] == -1:
            portfolio.loc[i, 'amount_usd'] = portfolio.loc[i-1, 'amount_btc'] * portfolio.loc[i, 'close'] # Sell BTC
            portfolio.loc[i, 'amount_btc'] = 0
        elif df['signal'].iloc[i] == 0:
            portfolio.loc[i, 'amount_btc'] = portfolio.loc[i-1, 'amount_btc']
            portfolio.loc[i, 'amount_usd'] = portfolio.loc[i-1, 'amount_usd']
        
        portfolio.loc[i, 'value'] = portfolio.loc[i, 'amount_usd'] + portfolio.loc[i, 'amount_btc'] * portfolio.loc[i, 'close']
        
        if portfolio.loc[i, 'value'] < portfolio.loc[i-1, 'min_value']:
            portfolio.loc[i, 'min_value'] = portfolio.loc[i, 'value']
        else:
            portfolio.loc[i, 'min_value'] = portfolio.loc[i-1, 'min_value']
            
    portfolio['pnl'] = (portfolio['value']-10000)/10000
    portfolio['port_max_dd'] = (portfolio['min_value']-10000)/10000
            
    return portfolio

In [40]:
"""#Example
calculate_portfolio(signal_sma(date_filter(price, '2020-09-01', '2021-08-30'), 20, 60))"""

"#Example\ncalculate_portfolio(signal_sma(date_filter(price, '2020-09-01', '2021-08-30'), 20, 60))"

# 6. Optimize the Strategies

In [41]:
def experiment_sma(df, ls, hs, ll, hl, pnl, dd, start_date, end_date):
    experiment = pd.DataFrame()
    p = 0
    
    for j in range(ll, hl + 1):
        for k in range(ls, hs + 1):
            if j > k:
                port = calculate_portfolio(signal_sma(date_filter(df, start_date, end_date), k, j))
                
                if port['port_max_dd'].iloc[len(port) - 1] > dd and port['pnl'].iloc[len(port) - 1] > pnl:
                
                    experiment.loc[p, 'short_sma'] = k
                    experiment.loc[p, 'long_sma'] = j
                    experiment.loc[p, 'pnl'] = port['pnl'].iloc[len(port) - 1]
                    experiment.loc[p, 'port_max_dd'] = port['port_max_dd'].iloc[len(port) - 1]
                    p = p + 1
                    
    experiment.to_excel('C:/Users/54115/Documents/Ramiro/Taller/experiment_sma_all.xlsx', index = False)
                
    return experiment

In [42]:
%%time
experiment_sma(price, 15, 65, 25, 85, 0, -0.4, '2018-01-01', '2023-06-05')

CPU times: total: 42min 2s
Wall time: 42min 25s


,short_sma,long_sma,pnl,port_max_dd
0,15.0,25.0,1.323271,-0.286653
1,16.0,25.0,1.236336,-0.277663
2,17.0,25.0,1.542112,-0.226808
3,18.0,25.0,0.783143,-0.234976
4,19.0,25.0,0.704721,-0.278047
...,...,...,...,...
998,61.0,85.0,1.782843,-0.266278
999,62.0,85.0,1.407154,-0.295007
1000,63.0,85.0,1.402560,-0.266855
1001,64.0,85.0,1.799574,-0.317034


In [43]:
def experiment_ema(df, ls, hs, ll, hl, pnl, dd, start_date, end_date):
    experiment = pd.DataFrame()
    p = 0
    
    for j in range(ll, hl + 1):
        for k in range(ls, hs + 1):
            if j > k:
                port = calculate_portfolio(signal_ema(date_filter(df, start_date, end_date), k, j))
                
                if port['port_max_dd'].iloc[len(port) - 1] > dd and port['pnl'].iloc[len(port) - 1] > pnl:
                
                    experiment.loc[p, 'short_ema'] = k
                    experiment.loc[p, 'long_ema'] = j
                    experiment.loc[p, 'pnl'] = port['pnl'].iloc[len(port) - 1]
                    experiment.loc[p, 'port_max_dd'] = port['port_max_dd'].iloc[len(port) - 1]
                    p = p + 1
                    
    experiment.to_excel('C:/Users/54115/Documents/Ramiro/Taller/experiment_ema_all.xlsx', index = False)
                
    return experiment

In [44]:
%%time
experiment_ema(price, 15, 65, 25, 85, 0, -0.4, '2018-01-01', '2023-06-05')

CPU times: total: 42min 28s
Wall time: 42min 43s


,short_ema,long_ema,pnl,port_max_dd
0,27.0,29.0,5.285490,-0.395945
1,28.0,29.0,5.105305,-0.341558
2,26.0,30.0,5.438574,-0.395945
3,27.0,30.0,5.105305,-0.341558
4,28.0,30.0,5.105130,-0.351624
...,...,...,...,...
1932,61.0,85.0,2.535337,-0.240963
1933,62.0,85.0,2.535337,-0.240963
1934,63.0,85.0,2.602817,-0.220678
1935,64.0,85.0,2.625202,-0.221393


In [45]:
def experiment_boll(df, lm, hm, lo, ho, pnl, dd, start_date, end_date):
    experiment = pd.DataFrame()
    p = 0
    
    for j in range(lm, hm + 1):
        for k in [round(num, 1) for num in list(float(x) / 10 for x in range(int(10 * lo), int(10 * ho + 1)))]:
            port = calculate_portfolio(signal_boll(date_filter(df, start_date, end_date), j, k))
                
            if port['port_max_dd'].iloc[len(port) - 1] > dd and port['pnl'].iloc[len(port) - 1] > pnl:
                
                experiment.loc[p, 'ma'] = j
                experiment.loc[p, 'stdev'] = k
                experiment.loc[p, 'pnl'] = port['pnl'].iloc[len(port) - 1]
                experiment.loc[p, 'port_max_dd'] = port['port_max_dd'].iloc[len(port) - 1]
                p = p + 1
                
    experiment.to_excel('C:/Users/54115/Documents/Ramiro/Taller/experiment_boll_all.xlsx', index = False)
                
    return experiment

In [46]:
%%time
experiment_boll(price, 8, 30, 1, 2.8, 0, -0.4, '2018-01-01', '2023-06-05')

CPU times: total: 8min 29s
Wall time: 8min 33s


,ma,stdev,pnl,port_max_dd
0,18.0,1.0,0.249229,-0.364988
1,18.0,1.1,0.249229,-0.364988
2,18.0,1.2,0.249229,-0.364988
3,18.0,1.3,0.249229,-0.364988
4,18.0,1.4,0.249229,-0.364988
5,18.0,1.5,0.249229,-0.364988
6,18.0,1.6,0.249229,-0.364988
7,18.0,1.7,0.249229,-0.364988
8,18.0,1.8,0.249229,-0.364988
9,18.0,1.9,0.249229,-0.364988


In [47]:
def experiment_rsi(df, lm, hm, ll, hl, lh, hh, pnl, dd, start_date, end_date):
    experiment = pd.DataFrame()
    p = 0
    
    for j in range(lm, hm + 1):
        for k in range(ll,hl + 1):
            for z in range(lh, hh + 1):
                port = calculate_portfolio(signal_rsi(date_filter(df, start_date, end_date), j, k, z))

                if port['port_max_dd'].iloc[len(port) - 1] > dd and port['pnl'].iloc[len(port) - 1] > pnl:

                    experiment.loc[p, 'periods'] = j
                    experiment.loc[p, 'oversold'] = k
                    experiment.loc[p, 'overbought'] = z
                    experiment.loc[p, 'pnl'] = port['pnl'].iloc[len(port) - 1]
                    experiment.loc[p, 'port_max_dd'] = port['port_max_dd'].iloc[len(port) - 1]
                    p = p + 1
    
    experiment.to_excel('C:/Users/54115/Documents/Ramiro/Taller/experiment_rsi_all.xlsx', index = False)
    
    return experiment

In [48]:
%%time
experiment_rsi(price, 6, 25, 15, 40, 60, 85, 0, -0.4, '2018-01-01', '2023-06-05')

CPU times: total: 4h 5min 19s
Wall time: 4h 6min 54s


,periods,oversold,overbought,pnl,port_max_dd
0,6.0,15.0,60.0,1.699711,-0.218530
1,6.0,15.0,61.0,1.519620,-0.218530
2,6.0,15.0,62.0,1.356944,-0.218530
3,6.0,15.0,63.0,1.530970,-0.218530
4,6.0,15.0,64.0,1.799113,-0.218530
...,...,...,...,...,...
2569,25.0,35.0,61.0,0.261369,-0.207880
2570,25.0,35.0,62.0,0.432722,-0.145692
2571,25.0,35.0,64.0,0.314016,-0.243786
2572,25.0,35.0,65.0,0.327377,-0.236097


In [49]:
def experiment_macd(df, ls, hs, ll, hl, lt, ht, pnl, dd, start_date, end_date):
    experiment = pd.DataFrame()
    p = 0
    
    for j in range(ls, hs):
        for k in range(ll, hl):
            if k > j:
                for z in range(lt, ht):
                    port = calculate_portfolio(signal_macd(date_filter(df, start_date, end_date), j, k, z))

                    if port['port_max_dd'].iloc[len(port) - 1] > dd and port['pnl'].iloc[len(port) - 1] > pnl:

                        experiment.loc[p, 'short_ema'] = j
                        experiment.loc[p, 'long_ema'] = k
                        experiment.loc[p, 'signal'] = z
                        experiment.loc[p, 'pnl'] = port['pnl'].iloc[len(port) - 1]
                        experiment.loc[p, 'port_max_dd'] = port['port_max_dd'].iloc[len(port) - 1]
                        p = p + 1
                    
    experiment.to_excel('C:/Users/54115/Documents/Ramiro/Taller/experiment_macd_all.xlsx', index = False)
                
    return experiment

In [50]:
%%time
experiment_macd(price, 8, 20, 17, 31, 5, 13, 0, -0.4, '2018-01-01', '2023-06-05')

CPU times: total: 23min 13s
Wall time: 23min 23s


,short_ema,long_ema,signal,pnl,port_max_dd
0,8.0,17.0,5.0,2.351305,-0.390047
1,8.0,17.0,12.0,2.004331,-0.306233
2,8.0,18.0,11.0,2.091955,-0.304263
3,8.0,18.0,12.0,2.717445,-0.282570
4,8.0,19.0,11.0,2.814373,-0.303959
...,...,...,...,...,...
1041,19.0,30.0,8.0,2.304867,-0.181918
1042,19.0,30.0,9.0,4.170540,-0.149059
1043,19.0,30.0,10.0,4.055630,-0.132789
1044,19.0,30.0,11.0,3.315759,-0.274920


# 7. Calculate the Portfolio from the Orders of a Manual Strategy

In [51]:
def calculate_manual_portfolio(price, port_name):
    """
    Calculate the evolution of the manual strategy portfolio and the portfolio metrics
    using the price and the signals of the manual strategy
    """
    
    price = price[['date', 'close', 'open', 'high', 'low']]
    
    #ETL for the orders Dataframe
    orders = pd.read_csv(direc + '/' + port_name + '.csv', 
                    sep = ',', 
                    usecols = [0,1,2,3], 
                    names = ['date', 'price', 'amount', 'type'],
                    header = 0)

    orders['date'] = pd.to_datetime(orders['date'], format='%d/%m/%Y')

    orders = orders.sort_values('date', ascending=True)

    orders['signal'] = np.where(orders['type']=='Compra', 1, -1)
    
    # Create the portfolio dataframe from price and orders
    portfolio = price.merge(orders,
                            left_on = 'date',
                            right_on = 'date',
                            how = 'left')
    
    portfolio['signal'].fillna(0, inplace = True)
    portfolio['signal'] = portfolio['signal'].astype(int)
    portfolio['amount'].fillna(0, inplace = True)
    
    portfolio['amount_btc'] = 0
    portfolio['amount_usd'] = 0
    portfolio['invested_usd'] = 0
    portfolio['value'] = 0
    
    for i in range(1, len(portfolio)):
        
        portfolio.loc[i, 'amount_btc'] = portfolio.loc[i, 'amount'] + portfolio.loc[i-1, 'amount_btc']
        
        if portfolio['signal'].iloc[i] == 1:
            if portfolio.loc[i, 'amount']*portfolio.loc[i, 'price'] >= portfolio.loc[i, 'amount_usd']:
                portfolio.loc[i, 'amount_usd'] = 0
                portfolio.loc[i, 'invested_usd'] = portfolio.loc[i, 'amount']*portfolio.loc[i, 'price'] + portfolio.loc[i-1, 'invested_usd'] - portfolio.loc[i-1, 'amount_usd']
            else:
                portfolio.loc[i, 'amount_usd'] = portfolio.loc[i-1, 'amount_usd'] - portfolio.loc[i, 'amount']*portfolio.loc[i, 'price']
                portfolio.loc[i, 'invested_usd'] = portfolio.loc[i-1, 'invested_usd']
                
        if portfolio['signal'].iloc[i] == -1:
            portfolio.loc[i, 'invested_usd'] = portfolio.loc[i-1, 'invested_usd']
            portfolio.loc[i, 'amount_usd'] = portfolio.loc[i-1, 'amount_usd'] - portfolio.loc[i, 'amount']*portfolio.loc[i, 'price']
                
        if portfolio['signal'].iloc[i] == 0:
            portfolio.loc[i, 'invested_usd'] = portfolio.loc[i-1, 'invested_usd']
            portfolio.loc[i, 'amount_usd'] = portfolio.loc[i-1, 'amount_usd']
            
        portfolio.loc[i, 'value'] = portfolio.loc[i, 'amount_usd'] + portfolio.loc[i, 'amount_btc'] * portfolio.loc[i, 'close']
        
    portfolio['pnl'] = (portfolio['value']-portfolio['invested_usd'])/portfolio['invested_usd']
            
    return portfolio[['date', 'close', 'price', 'amount', 'signal', 'amount_btc', 'amount_usd', 'invested_usd', 'value', 'pnl']]

In [52]:
port_list = ['microstrategy', 'el_salvador', 'tesla']

table = pd.DataFrame()
u = 0

for p in port_list:
    
    porta = calculate_manual_portfolio(price, p)
    
    table.loc[u, 'Strategy'] = p
    table.loc[u, 'pnl'] = porta.loc[len(porta)-1, 'pnl']
    
    u = u + 1
    
table.to_excel('C:/Users/54115/Documents/Ramiro/Taller/manual_strategies.xlsx', index = False)

In [53]:
table

,Strategy,pnl
0,microstrategy,-0.146851
1,el_salvador,-0.432756
2,tesla,-0.047557
